In [4]:
from aco import ACO
import numpy as np
from scipy.spatial import distance_matrix
import logging
from sklearn.preprocessing import StandardScaler

N_ANTS = 30
N_ITERATIONS = [1, 10, 30, 50, 100, 150, 200]



def heuristics_reevo(edge_attr):
    num_edges = edge_attr.shape[0]
    num_attributes = edge_attr.shape[1]

    heuristic_values = np.zeros_like(edge_attr)

    # Apply feature engineering on edge attributes
    transformed_attr = np.log1p(np.abs(edge_attr))  # Taking logarithm of absolute value of attributes
    
    # Normalize edge attributes
    scaler = StandardScaler()
    edge_attr_norm = scaler.fit_transform(transformed_attr)

    # Calculate correlation coefficients
    correlation_matrix = np.corrcoef(edge_attr_norm.T)

    # Calculate heuristic value for each edge attribute
    for i in range(num_edges):
        for j in range(num_attributes):
            if edge_attr_norm[i][j] != 0:
                heuristic_values[i][j] = np.exp(-8 * edge_attr_norm[i][j] * correlation_matrix[j][j])

    return heuristic_values


def solve(node_pos):
    dist_mat = distance_matrix(node_pos, node_pos)
    dist_mat[np.diag_indices_from(dist_mat)] = 1 # set diagonal to a large number
    heu = heuristics_reevo(dist_mat) + 1e-9
    heu[heu < 1e-9] = 1e-9
    aco = ACO(dist_mat, heu, n_ants=N_ANTS)
    
    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj = aco.run(N_ITERATIONS[i])
        else:
            obj = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        # print("Iteration: {}, Objective: {}".format(N_ITERATIONS[i], obj))
        results.append(obj.item())
    return results

In [5]:
print("[*] Running ...")

for problem_size in [20, 50, 100]:
    dataset_path = f"./dataset/test{problem_size}_dataset.npy"
    node_positions = np.load(dataset_path)
    logging.info(f"[*] Evaluating {dataset_path}")
    n_instances = node_positions.shape[0]
    objs = []
    for i, node_pos in enumerate(node_positions):
        obj = solve(node_pos)
        objs.append(obj)
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Running ...
[*] Average for 20, 1 iterations: 3.889379511456282
[*] Average for 20, 10 iterations: 3.835950148102565
[*] Average for 20, 30 iterations: 3.8290479032548324
[*] Average for 20, 50 iterations: 3.828073158679342
[*] Average for 20, 100 iterations: 3.8277813822143028
[*] Average for 20, 150 iterations: 3.827401936786191
[*] Average for 20, 200 iterations: 3.827357036991053

[*] Average for 50, 1 iterations: 6.297774555526493
[*] Average for 50, 10 iterations: 5.930767443576591
[*] Average for 50, 30 iterations: 5.856143165091699
[*] Average for 50, 50 iterations: 5.8347924836227865
[*] Average for 50, 100 iterations: 5.811928723670486
[*] Average for 50, 150 iterations: 5.801042592264493
[*] Average for 50, 200 iterations: 5.795809427231816

[*] Average for 100, 1 iterations: 9.39701402755209
[*] Average for 100, 10 iterations: 8.571086477028867
[*] Average for 100, 30 iterations: 8.335871168733245
[*] Average for 100, 50 iterations: 8.264958774055497
[*] Average for 100